In [5]:
import pytz
import re
import urllib
import pytz
import re
import json
import datetime
import requests
import logging


In [2]:
event_load = """{
    "AlarmName": "cab-prd-digitalbanking-rds-CPUUtilization-test-email",
    "AlarmDescription": null,
    "AWSAccountId": "984739340995",
    "AlarmConfigurationUpdatedTimestamp": "2024-03-20T07:01:35.846+0000",
    "NewStateValue": "ALARM",
    "NewStateReason": "Testing the Amazon Cloudwatch alarm",
    "StateChangeTime": "2024-03-20T07:01:50.985+0000",
    "Region": "Asia Pacific (Singapore)",
    "AlarmArn": "arn:aws:cloudwatch:ap-southeast-1:984739340995:alarm:cab-prd-digitalbanking-rds-CPUUtilization-test-email",
    "OldStateValue": "OK",
    "OKActions": [],
    "AlarmActions": [
        "arn:aws:sns:ap-southeast-1:984739340995:cab-prd-lambda-trigger-sns"
    ],
    "InsufficientDataActions": [],
    "Trigger": {
        "MetricName": "CPUUtilization",
        "Namespace": "AWS/RDS",
        "StatisticType": "Statistic",
        "Statistic": "AVERAGE",
        "Unit": null,
        "Dimensions": [
            {
                "value": "cab-prod-digital-db",
                "name": "DBInstanceIdentifier"
            }
        ],
        "Period": 60,
        "EvaluationPeriods": 1,
        "DatapointsToAlarm": 1,
        "ComparisonOperator": "GreaterThanOrEqualToThreshold",
        "Threshold": 80,
        "TreatMissingData": "missing",
        "EvaluateLowSampleCountPercentile": ""
    }
}
"""


In [6]:
TimeZoneCode = "Asia/Phnom_Penh"
TimezoneInitial = "UTC+7"

In [12]:

def changeAlarmToLocalTimeZone(event,timezoneCode,localTimezoneInitial):
    tz = pytz.timezone(timezoneCode)
    #exclude the Alarm event from the SNS records
    AlarmEvent = json.loads(event)

    #extract event data like alarm name, region, state, timestamp
    alarmName=AlarmEvent['AlarmName']
    descriptionexist=0
    if ("AlarmDescription" in AlarmEvent) and (AlarmEvent['AlarmDescription'] != None):
        description= AlarmEvent['AlarmDescription']
        descriptionexist=1
    reason=AlarmEvent['NewStateReason']
    region=AlarmEvent['Region']
    state=AlarmEvent['NewStateValue']
    previousState=AlarmEvent['OldStateValue']
    timestamp=AlarmEvent['StateChangeTime']
    alarmARN=AlarmEvent['AlarmArn']
    RegionID=alarmARN.split(":")[3]
    AccountID=AlarmEvent['AWSAccountId']

    #get the datapoints substring
    pattern = re.compile('\[(.*?)\]')
    
    #test if pattern match and there is datapoints
    if pattern.search(reason):
        Tempstr = pattern.findall(reason)[0]

        #get in the message all datapoints timestamps and convert to localTimezone using same format
        pattern = re.compile('\(.*?\)')
        m = pattern.finditer(Tempstr)
        for match in m:
            Tempstr=match.group()
            tempStamp = datetime.datetime.strptime(Tempstr, "(%d/%m/%y %H:%M:%S)")
            tempStamp = tempStamp.astimezone(tz)
            tempStamp = tempStamp.strftime('%d/%m/%y %H:%M:%S')
            reason=reason.replace(Tempstr, '('+tempStamp+')')
    

    #convert timestamp to localTimezone time
    timestamp = timestamp.split(".")[0]
    timestamp = datetime.datetime.strptime(timestamp, "%Y-%m-%dT%H:%M:%S")
    localTimeStamp = timestamp.astimezone(tz)
    localTimeStamp = localTimeStamp.strftime("%A %B, %Y %H:%M:%S")

    #create Custom message and change timestamps

    customMessage='You are receiving this email because your Amazon CloudWatch Alarm "'+alarmName+'" in the '+region+' region has entered the '+state+' state, at "'+localTimeStamp+' '+localTimezoneInitial +'.'
    
    # Add Console link
    customMessage=customMessage+'\n\n View this alarm in the AWS Management Console: \n'+ 'https://'+RegionID+'.console.aws.amazon.com/cloudwatch/home?region='+RegionID+'#s=Alarms&alarm='+urllib.parse.quote(alarmName)
    
    #Add Alarm Name
    customMessage=customMessage+'\n\n Alarm Details:\n- Name:\t\t\t\t\t\t'+alarmName
    
    # Add alarm description if exist
    if (descriptionexist == 1) : customMessage=customMessage+'\n- Description:\t\t\t\t\t'+description
    
    customMessage=customMessage+'\n- State Change:\t\t\t\t'+previousState+' -> '+state

    # Add alarm reason for changes
    customMessage=customMessage+'\n- Reason for State Change:\t\t'+reason
 
    # Add alarm evaluation timeStamp   
    customMessage=customMessage+'\n- Timestamp:\t\t\t\t\t'+localTimeStamp+' '+localTimezoneInitial

    # Add AccountID    
    customMessage=customMessage+'\n- AWS Account: \t\t\t\t'+AccountID
    
    # Add Alarm ARN
    customMessage=customMessage+'\n- Alarm Arn:\t\t\t\t\t'+alarmARN + '\U0001F973 This is a Robot face!\n'
    return customMessage

In [13]:
message = changeAlarmToLocalTimeZone(event_load,TimeZoneCode,TimezoneInitial)
print(message)

You are receiving this email because your Amazon CloudWatch Alarm "cab-prd-digitalbanking-rds-CPUUtilization-test-email" in the Asia Pacific (Singapore) region has entered the ALARM state, at "Wednesday March, 2024 07:01:50 UTC+7.

 View this alarm in the AWS Management Console: 
https://ap-southeast-1.console.aws.amazon.com/cloudwatch/home?region=ap-southeast-1#s=Alarms&alarm=cab-prd-digitalbanking-rds-CPUUtilization-test-email

 Alarm Details:
- Name:						cab-prd-digitalbanking-rds-CPUUtilization-test-email
- State Change:				OK -> ALARM
- Reason for State Change:		Testing the Amazon Cloudwatch alarm
- Timestamp:					Wednesday March, 2024 07:01:50 UTC+7
- AWS Account: 				984739340995
- Alarm Arn:					arn:aws:cloudwatch:ap-southeast-1:984739340995:alarm:cab-prd-digitalbanking-rds-CPUUtilization-test-email🥳 This is a Robot face!



In [3]:
# message = changeAlarmToLocalTimeZone(event_load,TimeZoneCode,TimezoneInitial)
TOKEN = "6763568873:AAF3L14zcl-_BPMrWxyZtGi5WA1zR1aks6Y"
USER_ID = "-4107871073"
TELEGRAM_URL = "https://api.telegram.org/bot{}/sendMessage".format(TOKEN)

In [4]:
def process_message(input):
    try:
        # Loading JSON into a string
        raw_json = json.loads(input)
        # Outputing as JSON with indents
        output = json.dumps(raw_json, indent=4)
    except:
        output = input
    return output

In [10]:
testing = json.dumps(event_load)
print(testing)

"{\n    \"AlarmName\": \"cab-prd-digitalbanking-rds-CPUUtilization-test-email\",\n    \"AlarmDescription\": null,\n    \"AWSAccountId\": \"984739340995\",\n    \"AlarmConfigurationUpdatedTimestamp\": \"2024-03-20T07:01:35.846+0000\",\n    \"NewStateValue\": \"ALARM\",\n    \"NewStateReason\": \"Testing the Amazon Cloudwatch alarm\",\n    \"StateChangeTime\": \"2024-03-20T07:01:50.985+0000\",\n    \"Region\": \"Asia Pacific (Singapore)\",\n    \"AlarmArn\": \"arn:aws:cloudwatch:ap-southeast-1:984739340995:alarm:cab-prd-digitalbanking-rds-CPUUtilization-test-email\",\n    \"OldStateValue\": \"OK\",\n    \"OKActions\": [],\n    \"AlarmActions\": [\n        \"arn:aws:sns:ap-southeast-1:984739340995:cab-prd-lambda-trigger-sns\"\n    ],\n    \"InsufficientDataActions\": [],\n    \"Trigger\": {\n        \"MetricName\": \"CPUUtilization\",\n        \"Namespace\": \"AWS/RDS\",\n        \"StatisticType\": \"Statistic\",\n        \"Statistic\": \"AVERAGE\",\n        \"Unit\": null,\n        \"Dim

In [11]:
# def lambda_handler(event, context):
    # logging the event for debugging
testing = json.dumps(event_load)
print(testing)
# Basic exception handling. If anything goes wrong, logging the exception    
try:
    # Reading the message "Message" field from the SNS message
    message = process_message(testing)

    # Payload to be set via POST method to Telegram Bot API
    payload = {
        "text": message.encode("utf8"),
        "chat_id": USER_ID
    }

    # Posting the payload to Telegram Bot API
    requests.post(TELEGRAM_URL, payload)

except Exception as e:
    raise e

"{\n    \"AlarmName\": \"cab-prd-digitalbanking-rds-CPUUtilization-test-email\",\n    \"AlarmDescription\": null,\n    \"AWSAccountId\": \"984739340995\",\n    \"AlarmConfigurationUpdatedTimestamp\": \"2024-03-20T07:01:35.846+0000\",\n    \"NewStateValue\": \"ALARM\",\n    \"NewStateReason\": \"Testing the Amazon Cloudwatch alarm\",\n    \"StateChangeTime\": \"2024-03-20T07:01:50.985+0000\",\n    \"Region\": \"Asia Pacific (Singapore)\",\n    \"AlarmArn\": \"arn:aws:cloudwatch:ap-southeast-1:984739340995:alarm:cab-prd-digitalbanking-rds-CPUUtilization-test-email\",\n    \"OldStateValue\": \"OK\",\n    \"OKActions\": [],\n    \"AlarmActions\": [\n        \"arn:aws:sns:ap-southeast-1:984739340995:cab-prd-lambda-trigger-sns\"\n    ],\n    \"InsufficientDataActions\": [],\n    \"Trigger\": {\n        \"MetricName\": \"CPUUtilization\",\n        \"Namespace\": \"AWS/RDS\",\n        \"StatisticType\": \"Statistic\",\n        \"Statistic\": \"AVERAGE\",\n        \"Unit\": null,\n        \"Dim

In [51]:
payload = {
            "text": message,
            "chat_id": USER_ID
        }

In [52]:
requests.post(TELEGRAM_URL, payload)

<Response [200]>